Cloning the repo from git

In [5]:
!pip install torch torchvision
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'yolov5'...
remote: Enumerating objects: 15705, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 15705 (delta 9), reused 19 (delta 4), pack-reused 15672
Receiving objects: 100% (15705/15705), 14.44 MiB | 10.60 MiB/s, done.
Resolving deltas: 100% (10755/10755), done.
/content/yolov5/yolov5/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Loading and configuring YOLOv5

In [6]:
import torch

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Set the model to the evaluation mode
model.eval()

# Configure YOLOv5 to detect humans (class index 0)
model.conf = 0.5  # Confidence threshold for object detection
model.classes = [0]  # Only detect humans
model.names = ['human']  # Set class name as 'human'

# Disable all other post-processing except bounding box visualization
model.autoscale = False  # Disable automatic scaling of bounding boxes
model.augment = False  # Disable test-time augmentation
model.visualize = True  # Enable bounding box visualization


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-5-23 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 333MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Defining CNN classifier

In [7]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

# Define your CNN classifier architecture
def create_classifier(input_shape, num_classes):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Set the number of classes/categories
num_classes = 3

# Set the dimensions of your input images
image_height = 128
image_width = 128

# Create an instance of your classifier
classifier = create_classifier((image_height, image_width, 3), num_classes)

# Compile the model
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [8]:
import cv2
import numpy as np

# Define a function to perform cropping and inference
def process_image(image, boxes, classifier):
    cropped_images = []
    predictions = []
    
    for box in boxes:
        x_min, y_min, x_max, y_max, _ = box
        x_min, y_min, x_max, y_max = int(x_min), int(y_min), int(x_max), int(y_max)
        
        # Crop the image based on the bounding box coordinates
        cropped_image = image[y_min:y_max, x_min:x_max]
        
        # Resize the cropped image to the input size of the classifier
        cropped_image = cv2.resize(cropped_image, (image_height, image_width))
        
        # Preprocess the cropped image (e.g., normalization, channel ordering)
        preprocessed_image = preprocess_input(cropped_image)
        
        # Append the cropped image to the list
        cropped_images.append(preprocessed_image)
    
    if cropped_images:
        # Convert the list of cropped images to a numpy array
        cropped_images = np.array(cropped_images)
        
        # Perform inference using the CNN classifier
        predictions = classifier.predict(cropped_images)
    
    return predictions

# Example usage:
image = cv2.imread('path/to/your/image.jpg')  # Load your input image
# boxes = [list of YOLOv5 bounding boxes]  # List of bounding boxes from YOLOv5
# predictions = process_image(image, boxes, classifier)  # Perform cropping and inference


In [9]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset directory
dataset_dir = '/content/drive/MyDrive/CNN'

# Set the desired input size for your classifier
input_size = (128, 128)

# Set the batch size for training and validation
batch_size = 32

# Set the number of classes
num_classes = 3

# Set the path to your training and validation data directories
train_dir = dataset_dir + '/train'
validation_dir = dataset_dir + '/test'

# Use data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Use rescaling for validation data
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load and preprocess training data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical',
    classes=['normal', 'dark']
)

# Load and preprocess validation data
validation_data = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Print the class labels
class_labels = list(train_data.class_indices.keys())
print("Class Labels:", class_labels)


In [ ]:
import cv2
import os
import numpy as np

# Set the path to your dataset directory
dataset_dir = '/content/drive/MyDrive/CNN'

# Set the desired input size for your classifier
input_size = (128, 128)

# Function to load and preprocess the dataset
def load_dataset():
    train_images = []
    train_labels = []

    # Load training images
    train_dir = os.path.join(dataset_dir, 'train')
    classes = os.listdir(train_dir)

    for class_idx, class_name in enumerate(classes):
        class_dir = os.path.join(train_dir, class_name)
        for image_file in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_file)
            image = cv2.imread(image_path)
            image = cv2.resize(image, input_size)
            image = image.astype(np.float32) / 255.0  # Normalize image
            train_images.append(image)
            train_labels.append(class_idx)

    return np.array(train_images), np.array(train_labels)

# Load and preprocess the dataset
train_images, train_labels = load_dataset()

# Print the shape of the loaded data
print('Train images shape:', train_images.shape)
print('Train labels shape:', train_labels.shape)


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Define the CNN classifier model
model = create_classifier(input_size, num_classes)

# Define the loss function and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Define the number of epochs and batch size
epochs = 10
batch_size = 32

# Train the model
history = model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=epochs, batch_size=batch_size)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels)

print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)


In [ ]:
# Define the path to save the trained weights
weights_path = '/content/drive/MyDrive/CNN/trained_classifier_weights.h5'

# Save the trained weights
model.save_weights(weights_path)

print('Trained classifier weights saved successfully.')


In [ ]:
import torch
import cv2
import numpy as np

# Set the path to trained classifier weights
classifier_weights_path = '/content/drive/MyDrive/CNN/trained_classifier_weights.h5'

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Load the CNN classifier model
classifier = create_classifier(input_size, num_classes)
classifier.load_weights(classifier_weights_path)

# Function to detect humans and classify their actions
def detect_and_classify(image):
    # Perform object detection using YOLOv5
    results = model(image)

    # Get the bounding box coordinates and confidence scores
    boxes = results.xyxy[0][:, :4]
    scores = results.xyxy[0][:, 4]

    # Filter out bounding boxes for humans based on confidence scores
    human_boxes = boxes[scores > 0.5]

    # Crop and classify each detected human image
    for box in human_boxes:
        xmin, ymin, xmax, ymax = map(int, box)

        # Crop the image based on the bounding box coordinates
        cropped_image = image[ymin:ymax, xmin:xmax]

        # Preprocess the cropped image for classification
        cropped_image = cv2.resize(cropped_image, input_size)
        cropped_image = cropped_image.astype(np.float32) / 255.0

        # Reshape the cropped image for input to the classifier
        cropped_image = np.expand_dims(cropped_image, axis=0)

        # Perform action classification using the CNN classifier
        action = classifier.predict(cropped_image)
        action_label = action.argmax()

        # Print the predicted action label
        print('Predicted action:', action_label)

# Load and preprocess the image for detection and classification
image_path = '/path/to/your/image.jpg'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = torch.from_numpy(image).unsqueeze(0).float() / 255.0

# Detect and classify actions in the image
detect_and_classify(image)


In [ ]:
import os
import matplotlib.pyplot as plt

# Set the path to the test images directory
test_images_dir = '/content/drive/MyDrive/CNN/normal/test'  # Update with your directory path

# Get the list of test images
test_images = os.listdir(test_images_dir)

# Function to plot the image with predicted action label
def plot_image(image_path, action_label):
    image = plt.imread(image_path)
    plt.imshow(image)
    plt.title('Predicted Action: {}'.format(action_label))
    plt.axis('off')
    plt.show()

# Loop through the test images and perform detection and classification
for image_name in test_images:
    image_path = os.path.join(test_images_dir, image_name)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = torch.from_numpy(image).unsqueeze(0).float() / 255.0

    # Detect and classify actions in the image
    detect_and_classify(image)

    # Plot the image with predicted action label
    plot_image(image_path, action_label)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, roc_curve, roc_auc_score
import seaborn as sns

# Function to compute precision-recall curve and plot it
def plot_precision_recall_curve(y_true, y_score):
    precision, recall, _ = precision_recall_curve(y_true, y_score)
    average_precision = average_precision_score(y_true, y_score)

    plt.figure()
    plt.step(recall, precision, color='b', alpha=0.2, where='post')
    plt.fill_between(recall, precision, step='post', alpha=0.2, color='b')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve: AP={0:0.2f}'.format(average_precision))
    plt.show()

# Function to compute and plot the confusion matrix
def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_norm, annot=True, cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

# Function to compute F1 confidence curve and plot it
def plot_f1_confidence_curve(y_true, y_score):
    f1_scores = []
    thresholds = np.arange(0.0, 1.05, 0.05)

    for threshold in thresholds:
        y_pred = (y_score >= threshold).astype(int)
        f1_scores.append(f1_score(y_true, y_pred))

    plt.plot(thresholds, f1_scores)
    plt.xlabel('Threshold')
    plt.ylabel('F1 Score')
    plt.title('F1 Confidence Curve')
    plt.show()

# Function to compute ROC curve and plot it
def plot_roc_curve(y_true, y_score):
    fpr, tpr, _ = roc_curve(y_true, y_score)
    roc_auc = roc_auc_score(y_true, y_score)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

# Function to classify actions and return predicted labels and confidence scores
def classify_actions(images):
    predicted_labels = []
    confidence_scores = []

    for image in images:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = torch.from_numpy(image).unsqueeze(0).float() / 255.0

        # Detect and classify actions in the image
        action = detect_and_classify(image)
        predicted_labels.append(action_label)
        confidence_scores.append(action.max())

    return predicted_labels, confidence_scores

# Load and preprocess the test images
test_images_dir = '/content/drive/MyDrive/CNN/normal/test'  # Update with your directory path
test_images = []

for image_name in os.listdir(test_images_dir):
    image_path = os.path.join(test_images_dir, image_name)
    image = cv2.imread(image_path)
    test_images.append(image)

# Convert the list of test images to numpy array
test_images = np.array(test_images)

# Classify actions and get predicted labels and confidence scores
predicted_labels, confidence_scores = classify_actions(test_images)

# Create a dictionary to map the action labels to numeric values
label_to_numeric = {'sit': 0, 'walk': 1, 'fall': 2}

# Convert the true labels to numeric values
true_labels = [label_to_numeric[os.path.dirname(image_name)] for image_name in os.listdir(test_images_dir)]

# Convert the predicted labels to numeric values
predicted_labels = [label_to_numeric[label] for label in predicted_labels]

# Compute and plot the precision-recall curve
plot_precision_recall_curve(true_labels, confidence_scores)

# Compute and plot the confusion matrix
classes = ['sit', 'walk', 'fall']
plot_confusion_matrix(true_labels, predicted_labels, classes)

# Compute and plot the F1 confidence curve
plot_f1_confidence_curve(true_labels, confidence_scores)

# Compute and plot the ROC curve
plot_roc_curve(true_labels, confidence_scores)